In [7]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

In [8]:
PINECONE_API_KEY = "pcsk_7YXVL5_BCBy3ZNdXqBgQMAT3FEFdVFdRrVuHEPh2Eofk1ntE7A91vqbdNDCumeE5nkhhMF"
PINECONE_API_ENV = "gcp-starter"

In [9]:
def load_pdf(data):
    loaser = DirectoryLoader(data,
                            glob="*.pdf",
                            oader_cls=PyPDFLoader) 

    documents = loader.load()
    
    return documents